In [19]:
%load_ext nb_black
# !pip install nb_black
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [20]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import metrics as m
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE  # doctest: +NORMALIZE_WHITESPACE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import ColumnSelector
from sklearn import model_selection
from mlxtend.classifier import StackingClassifier
from thundersvm import SVC as svmgpu
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE, SMOTENC  # doctest: +NORMALIZE_WHITESPACE
from sklearn.model_selection import cross_val_score
from numpy import average
import xgboost as xgb
from sklearn import metrics as m
import itertools

<IPython.core.display.Javascript object>

In [21]:
randomseed = 7
np.random.seed(randomseed)

<IPython.core.display.Javascript object>

In [22]:
# Rf_list

<IPython.core.display.Javascript object>

 2.Read dataset
 ===

In [23]:
# read the data set
x_original = pd.read_csv("../../../dataset/XLable_onlyDiabeticRemoved.txt")

conditions = [
    (x_original.L100800 < 100),
    (x_original.L100800 >= 100) & (x_original.L100800 < 126),
    (x_original.L100800 >= 126),
]
choices = [0, 1, 2]
x_original["CLASS"] = np.select(conditions, choices, default=0)
x_original = x_original[
    [
        "Unnamed: 0",
        "L100800",
        "L104600",
        "L103000",
        "S000300",
        "L101700",
        "L100700",
        "FIELD_33",
        "FIELD_38",
        "FIELD_40",
        "FIELD_31",
        "SEX",
        "AGE",  #'CLASS',
        "FIELD_16",
        "FIELD_23",
        "FIELD_15",
        "FIELD_22",
        "FIELD_17",
        "FIELD_24",
    ]
]

print(x_original.shape)

y_original = pd.read_csv("../../../dataset/TargetLable_onlyDiabeticRemoved.txt")

conditions = [
    (y_original.L100800 < 100),
    (y_original.L100800 >= 100) & (y_original.L100800 < 126),
    (y_original.L100800 >= 126),
]

choices = [0, 1, 2]
y_original["CLASS"] = np.select(conditions, choices, default=0)

y_original = y_original[["Unnamed: 0", "CLASS"]]

print(y_original.shape)

data = pd.merge(
    x_original, y_original, how="inner", left_on="Unnamed: 0", right_on="Unnamed: 0"
)



# filter the data set
data = data[data.FIELD_16 != 1]  # exclude people who are diagnosed for (diabetes)
data = data[data.FIELD_23 != 1]  # exclude people who are on medication for diabetes

data = data[
    data.FIELD_15 != 1
]  # exclude people who are diagnosed for (high blood pressure)
data = data[
    data.FIELD_22 != 1
]  # exclude people who are on medication for high blood pressure

data = data[data.FIELD_17 != 1]  # exclude people who are diagnosed for hyperlipidemia
data = data[
    data.FIELD_24 != 1
]  # exclude people who are on medication for hyperlipidemia

print(data.shape)

data = data[
    [
        "L100800",
        "L104600",
        "L103000",
        "S000300",
        "L101700",
        "L100700",
        "FIELD_33",
        "FIELD_38",
        "FIELD_40",
        "FIELD_31",
        "SEX",
        "AGE",
        "CLASS",
    ]
]
data = data.dropna()
print(data.shape)
data.head()



(185843, 19)
(185843, 2)
(169024, 20)
(56542, 13)


,L100800,L104600,L103000,S000300,L101700,L100700,FIELD_33,FIELD_38,FIELD_40,FIELD_31,SEX,AGE,CLASS
2,78.0,5.28,41.0,20.2,15.0,3.8,1.0,2.0,1.0,0.0,1.0,46.0,0
5,90.0,5.74,50.0,25.5,12.0,3.4,1.0,0.0,1.0,0.0,1.0,52.0,0
10,86.0,5.83,45.0,21.2,17.0,3.9,1.0,0.0,1.0,1.0,1.0,37.0,0
11,86.0,4.73,54.0,22.0,30.0,4.2,1.0,2.0,3.0,0.0,1.0,39.0,0
20,87.0,5.60,340.0,24.6,26.0,4.7,1.0,0.0,2.0,0.0,1.0,59.0,0


<IPython.core.display.Javascript object>

In [24]:
diabetic = data[data.CLASS == 2]
prediabetic = data[data.CLASS == 1]
normal = data[data.CLASS == 0]

print(diabetic.shape[0], prediabetic.shape[0], normal.shape[0])

1045 17331 38166


<IPython.core.display.Javascript object>

In [25]:
diabetic_test = diabetic.sample(200, random_state=randomseed)
prediabetic_test = prediabetic.sample(200, random_state=randomseed)
normal_test = normal.sample(200, random_state=randomseed)
test = pd.concat([diabetic_test, prediabetic_test, normal_test])

diabetic_train = diabetic.drop(diabetic_test.index)
prediabetic_train = prediabetic.drop(prediabetic_test.index)
# .sample(
#     10 * diabetic_train.shape[0], random_state=randomseed
# )
normal_train = normal.drop(normal_test.index).sample(
    prediabetic_train.shape[0],
    random_state=randomseed
    #     10 * diabetic_train.shape[0], random_state=randomseed
)
train = pd.concat([diabetic_train, diabetic_train, prediabetic_train, normal_train])

<IPython.core.display.Javascript object>

In [26]:
xtrain = train.iloc[:, :-1]
ytrain = train.iloc[:, -1]
xtest = test.iloc[:, :-1]
ytest = test.iloc[:, -1]

<IPython.core.display.Javascript object>

In [27]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

<IPython.core.display.Javascript object>

In [28]:
from imblearn.over_sampling import SMOTE, SMOTENC  # doctest: +NORMALIZE_WHITESPACE

randomseed = 42

sm = SMOTE(random_state=randomseed, sampling_strategy="minority")
X_res, y_res = sm.fit_resample(xtrain, ytrain)

print("Resampled dataset shape %s" % Counter(y_res))
print(
    y_res[y_res == 0].shape[0], y_res[y_res == 1].shape[0], y_res[y_res == 2].shape[0]
)
print(X_res.shape, y_res.shape)

xtrain = X_res
ytrain = y_res

Resampled dataset shape Counter({2: 17131, 1: 17131, 0: 17131})
17131 17131 17131
(51393, 12) (51393,)


<IPython.core.display.Javascript object>

3.Train and Generate models 
===

In [29]:
weakmodles = []
estimators = []


<IPython.core.display.Javascript object>

In [30]:
def changelist(strlist):
    test_list = (strlist).replace("[", "").replace("]", "").replace("'", "").split(",")
    test_list = [int(i) for i in test_list]
    return test_list

<IPython.core.display.Javascript object>

In [32]:
rf_12 = RandomForestClassifier(
    random_state=randomseed,
    n_estimators=150,
    max_depth=30,
    min_samples_split=2,
    min_samples_leaf=10,
    max_features="auto",
)


rf_12_pip = make_pipeline(ColumnSelector(cols=[0, 1, 3, 8, 9, 10, 11]), rf_12)

xgb_model_12 = xgb.XGBClassifier(random_state=0, max_depth=8, n_estimators=20)
xgb_model_12_pip = make_pipeline(
    ColumnSelector(cols=[0, 1, 3, 8, 9, 10, 11]), xgb_model_12
)


from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

# scv_12 = svmgpu(
#     C=100,
#     cache_size=None,
#     class_weight={},
#     coef0=0.0,
#     decision_function_shape="ovo",
#     degree=3,
#     gamma=0.1,
#     gpu_id=0,
#     kernel="linear",
#     max_iter=-1,
#     max_mem_size=-1,
#     n_jobs=-1,
#     probability=True,
#     random_state=None,
#     shrinking=False,
#     tol=0.001,
#     verbose=False,
# )

<IPython.core.display.Javascript object>

4.Train ensemble modles
===

4.1.CIM
===

In [33]:
weakmodles = [rf_12, xgb_model_12, rf_12_pip, xgb_model_12_pip]
estimators = [
    ("rf_12", rf_12),
    ("xgb_model_12", xgb_model_12),
    ("rf_12_pip", rf_12_pip),
    ("xgb_model_12_pip", xgb_model_12_pip),
]

<IPython.core.display.Javascript object>

In [34]:
acclist = []
predprobalist = []
predprobalist2 = []
confmatrxlist = []
ypredconfprob_all = []

<IPython.core.display.Javascript object>

In [35]:
counter = 0
for f in weakmodles:

    print(counter)
    counter = counter + 1

    rfclf = f
    rfclf.fit(xtrain, ytrain)
    ypred = rfclf.predict(xtest)
    ypredprob = rfclf.predict_proba(xtest)

    accuracy = m.accuracy_score(ytest, ypred)
    acclist.append(accuracy)
    predprobalist.append(ypredprob)
    predprobalist2.append(ypred)

    confmat = m.confusion_matrix(ytest, ypred)
    confmatrxlist.append(confmat)

    confsumh = np.sum(confmat, axis=1)
    propconfmat = confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
    ypredconfprob_all.append(propconfmat / 100)


cimc2 = np.zeros((ytest.shape[0], 3))
# # for each classifier
for j in range(len(weakmodles)):
    #     #for each data point
    cimc = []
    for i in range(ytest.shape[0]):
        # print(i,ypredconfprob_all[j],predprobalist[j][i])

        c1 = np.sum(predprobalist[j][i] * ypredconfprob_all[j][0])
        c2 = np.sum(predprobalist[j][i] * ypredconfprob_all[j][1])
        c3 = np.sum(predprobalist[j][i] * ypredconfprob_all[j][2])

        # print([c1,c2,c3])
        cimc.append([c1, c2, c3])

    cimc = np.array(cimc)
    ypred2 = np.argmax(cimc, axis=1)
    print((m.accuracy_score(ytest, ypred2)))
    cimc2 = cimc2 + cimc

print("final cim ==== >")
cimc2 = np.array(cimc2)
ypred3 = np.argmax(cimc2, axis=1)
print((m.accuracy_score(ytest, ypred3)))

0
1
2
3
0.725
0.7166666666666667
0.73
0.7316666666666667
final cim ==== >
0.73


<IPython.core.display.Javascript object>

In [36]:
print("final cim ==== >")
cimc2 = np.array(cimc2)
ypred3 = np.argmax(cimc2, axis=1)
print((m.accuracy_score(ytest, ypred3)))

final cim ==== >
0.73


<IPython.core.display.Javascript object>

In [37]:
confmatrx = pd.DataFrame(
    m.confusion_matrix(ytest, ypred3),
    columns=["Normal", "Prediabetes", "diabetes"],
    index=["Normal", "Prediabetes", "diabetes"],
)
confmatrx.head()

,Normal,Prediabetes,diabetes
Normal,152,48,0
Prediabetes,60,120,20
diabetes,4,30,166


<IPython.core.display.Javascript object>

In [38]:
print(m.classification_report(ytest, ypred3))

              precision    recall  f1-score   support

           0       0.70      0.76      0.73       200
           1       0.61      0.60      0.60       200
           2       0.89      0.83      0.86       200

    accuracy                           0.73       600
   macro avg       0.73      0.73      0.73       600
weighted avg       0.73      0.73      0.73       600



<IPython.core.display.Javascript object>

cross val
===

In [39]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, random_state=randomseed, shuffle=True)
kf.get_n_splits(xtrain)

print(kf)

data_cv = []
cv_acc = []
cv_precision = []
cv_recall = []
cv_f1 = []

counter = 0
for train_index, test_index in kf.split(xtrain):
    # print("TRAIN:", train_index, "TEST:", test_index)
    cv_xtrain, cv_xtest = xtrain[train_index], xtrain[test_index]
    cv_ytrain, cv_ytest = ytrain[train_index], ytrain[test_index]

    acclist = []
    predprobalist = []
    predprobalist2 = []
    confmatrxlist = []
    ypredconfprob_all = []

    print(counter)
    counter = counter + 1

    for f in weakmodles:

        rfclf = f
        rfclf.fit(cv_xtrain, cv_ytrain)
        ypred = rfclf.predict(cv_xtest)
        ypredprob = rfclf.predict_proba(cv_xtest)

        accuracy = m.accuracy_score(cv_ytest, ypred)
        acclist.append(accuracy)
        predprobalist.append(ypredprob)
        predprobalist2.append(ypred)

        confmat = m.confusion_matrix(cv_ytest, ypred)
        confmatrxlist.append(confmat)

        confsumh = np.sum(confmat, axis=1)
        propconfmat = confmat.copy()
        for i in range(propconfmat.shape[0]):
            propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
        ypredconfprob_all.append(propconfmat / 100)

    cimc2 = np.zeros((cv_ytest.shape[0], 3))
    # # for each classifier
    for j in range(len(weakmodles)):
        #     #for each data point
        cimc = []
        for i in range(cv_ytest.shape[0]):
            # print(i,ypredconfprob_all[j],predprobalist[j][i])

            c1 = np.sum(predprobalist[j][i] * ypredconfprob_all[j][0])
            c2 = np.sum(predprobalist[j][i] * ypredconfprob_all[j][1])
            c3 = np.sum(predprobalist[j][i] * ypredconfprob_all[j][2])

            # print([c1,c2,c3])
            cimc.append([c1, c2, c3])

        cimc = np.array(cimc)
        ypred2 = np.argmax(cimc, axis=1)
        print((m.accuracy_score(cv_ytest, ypred2)))
        cimc2 = cimc2 + cimc

    print("final cim ==== >")
    cimc2 = np.array(cimc2)
    ypred3 = np.argmax(cimc2, axis=1)
    print((m.accuracy_score(cv_ytest, ypred3)))

    cv_acc.append((m.accuracy_score(cv_ytest, ypred3)))
    cv_precision.append((m.precision_score(cv_ytest, ypred3, average="weighted")))
    cv_recall.append((m.recall_score(cv_ytest, ypred3, average="weighted")))
    cv_f1.append((m.f1_score(cv_ytest, ypred3, average="weighted")))

KFold(n_splits=10, random_state=42, shuffle=True)
0
0.7737354085603113
0.7795719844357977
0.7702334630350195
0.7791828793774319
final cim ==== >
0.7799610894941634
1
0.7819066147859922
0.7801556420233463
0.7671206225680934
0.7708171206225681
final cim ==== >
0.7836575875486381
2
0.7852140077821012
0.791828793774319
0.7735408560311284
0.7885214007782101
final cim ==== >
0.7908560311284046
3
0.7736913796458454
0.7795290912628916
0.7676590776415645
0.7711617046117922
final cim ==== >
0.7779723681650127
4
0.7880910683012259
0.7915936952714536
0.7771940066160732
0.7810858143607706
final cim ==== >
0.78945320101187
5
0.7880910683012259
0.7927612375948628
0.772913018096906
0.7803074528118311
final cim ==== >
0.78945320101187
6
0.778945320101187
0.7793345008756567
0.766880716092625
0.7752481027437245
final cim ==== >
0.7773885970033081
7
0.7787507297139521
0.7803074528118311
0.7731076084841408
0.7773885970033081
final cim ==== >
0.7816695855224752
8
0.7840046701692936
0.7898423817863398
0.7699

<IPython.core.display.Javascript object>

In [40]:
print(np.mean(cv_acc), " +/- ", np.std(cv_acc))
print(np.mean(cv_precision), " +/- ", np.std(cv_precision))
print(np.mean(cv_recall), " +/- ", np.std(cv_recall))
print(np.mean(cv_f1), " +/- ", np.std(cv_f1))

0.7846399207100959  +/-  0.004864798211224427
0.7800153156275351  +/-  0.005099740386223017
0.7846399207100959  +/-  0.004864798211224427
0.7797836612085387  +/-  0.005349165015689856


<IPython.core.display.Javascript object>

In [57]:
cv_acc

[0.7799610894941634,
 0.7836575875486381,
 0.7908560311284046,
 0.7779723681650127,
 0.78945320101187,
 0.78945320101187,
 0.7773885970033081,
 0.7816695855224752,
 0.7875072971395213,
 0.7884802490756957]

<IPython.core.display.Javascript object>

In [58]:
cv_precision

[0.7749645863369719,
 0.779856360665907,
 0.7867829872294209,
 0.7730762379010668,
 0.7848578093227923,
 0.7844284063039038,
 0.772617670326658,
 0.7761076936551519,
 0.7839013373697553,
 0.7835600671637242]

<IPython.core.display.Javascript object>

In [59]:
cv_recall

[0.7799610894941634,
 0.7836575875486381,
 0.7908560311284046,
 0.7779723681650127,
 0.78945320101187,
 0.78945320101187,
 0.7773885970033081,
 0.7816695855224752,
 0.7875072971395213,
 0.7884802490756957]

<IPython.core.display.Javascript object>

In [60]:
cv_f1

[0.7751509135515129,
 0.7789412910213099,
 0.7867405504679623,
 0.7719863528100721,
 0.7853714647214599,
 0.7852480405432741,
 0.7718285898978403,
 0.7766198125236584,
 0.7830857027850595,
 0.7828638937632374]

<IPython.core.display.Javascript object>

4.2.Stacking approach
==

In [41]:
# stacking classifier
sclf = StackingClassifier(
    classifiers=weakmodles,
    verbose=2,
    meta_classifier=RandomForestClassifier(n_estimators=500),
)
sclf.fit(xtrain, ytrain)
ypred = sclf.predict((xtest))


Fitting 4 classifiers...
Fitting classifier1: randomforestclassifier (1/4)
RandomForestClassifier(max_depth=30, min_samples_leaf=10, n_estimators=150,
                       random_state=42)
Fitting classifier2: xgbclassifier (2/4)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=20, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Fitting classifier3: pipeline (3/4)
Pipeline(steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 8, 9, 10, 11])),
       

<IPython.core.display.Javascript object>

In [42]:
print("accuracy \t", m.accuracy_score(ytest, ypred))
confmatrx = pd.DataFrame(
    m.confusion_matrix(ytest, ypred),
    columns=["Normal", "Prediabetes", "diabetes"],
    index=["Normal", "Prediabetes", "diabetes"],
)
confmatrx.head()

accuracy 	 0.7233333333333334


,Normal,Prediabetes,diabetes
Normal,150,50,0
Prediabetes,56,135,9
diabetes,4,47,149


<IPython.core.display.Javascript object>

In [43]:
print(m.classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.71      0.75      0.73       200
           1       0.58      0.68      0.62       200
           2       0.94      0.74      0.83       200

    accuracy                           0.72       600
   macro avg       0.75      0.72      0.73       600
weighted avg       0.75      0.72      0.73       600



<IPython.core.display.Javascript object>

In [44]:
scores = model_selection.cross_val_score(
    sclf, xtrain, ytrain, cv=10, scoring="accuracy"
)
print(
    "Accuracy: %0.4f (+/- %0.4f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "SVC_clf", scores)
)

Fitting 4 classifiers...
Fitting classifier1: randomforestclassifier (1/4)
RandomForestClassifier(max_depth=30, min_samples_leaf=10, n_estimators=150,
                       random_state=42)
Fitting classifier2: xgbclassifier (2/4)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=20, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Fitting classifier3: pipeline (3/4)
Pipeline(steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 8, 9, 10, 11])),
       

Fitting 4 classifiers...
Fitting classifier1: randomforestclassifier (1/4)
RandomForestClassifier(max_depth=30, min_samples_leaf=10, n_estimators=150,
                       random_state=42)
Fitting classifier2: xgbclassifier (2/4)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=20, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Fitting classifier3: pipeline (3/4)
Pipeline(steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 8, 9, 10, 11])),
       

Fitting 4 classifiers...
Fitting classifier1: randomforestclassifier (1/4)
RandomForestClassifier(max_depth=30, min_samples_leaf=10, n_estimators=150,
                       random_state=42)
Fitting classifier2: xgbclassifier (2/4)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=20, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Fitting classifier3: pipeline (3/4)
Pipeline(steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 8, 9, 10, 11])),
       

<IPython.core.display.Javascript object>

In [45]:
scores = model_selection.cross_val_score(
    sclf, xtrain, ytrain, cv=10, scoring="precision_weighted"
)
print(
    "precision: %0.4f (+/- %0.4f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "SVC_clf", scores)
)

Fitting 4 classifiers...
Fitting classifier1: randomforestclassifier (1/4)
RandomForestClassifier(max_depth=30, min_samples_leaf=10, n_estimators=150,
                       random_state=42)
Fitting classifier2: xgbclassifier (2/4)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=20, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Fitting classifier3: pipeline (3/4)
Pipeline(steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 8, 9, 10, 11])),
       

Fitting 4 classifiers...
Fitting classifier1: randomforestclassifier (1/4)
RandomForestClassifier(max_depth=30, min_samples_leaf=10, n_estimators=150,
                       random_state=42)
Fitting classifier2: xgbclassifier (2/4)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=20, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Fitting classifier3: pipeline (3/4)
Pipeline(steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 8, 9, 10, 11])),
       

Fitting 4 classifiers...
Fitting classifier1: randomforestclassifier (1/4)
RandomForestClassifier(max_depth=30, min_samples_leaf=10, n_estimators=150,
                       random_state=42)
Fitting classifier2: xgbclassifier (2/4)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=20, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Fitting classifier3: pipeline (3/4)
Pipeline(steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 8, 9, 10, 11])),
       

<IPython.core.display.Javascript object>

In [46]:
scores = model_selection.cross_val_score(
    sclf, xtrain, ytrain, cv=10, scoring="recall_weighted"
)
print(
    "recall: %0.4f (+/- %0.4f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "SVC_clf", scores)
)

Fitting 4 classifiers...
Fitting classifier1: randomforestclassifier (1/4)
RandomForestClassifier(max_depth=30, min_samples_leaf=10, n_estimators=150,
                       random_state=42)
Fitting classifier2: xgbclassifier (2/4)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=20, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Fitting classifier3: pipeline (3/4)
Pipeline(steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 8, 9, 10, 11])),
       

Fitting 4 classifiers...
Fitting classifier1: randomforestclassifier (1/4)
RandomForestClassifier(max_depth=30, min_samples_leaf=10, n_estimators=150,
                       random_state=42)
Fitting classifier2: xgbclassifier (2/4)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=20, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Fitting classifier3: pipeline (3/4)
Pipeline(steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 8, 9, 10, 11])),
       

Fitting 4 classifiers...
Fitting classifier1: randomforestclassifier (1/4)
RandomForestClassifier(max_depth=30, min_samples_leaf=10, n_estimators=150,
                       random_state=42)
Fitting classifier2: xgbclassifier (2/4)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=20, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Fitting classifier3: pipeline (3/4)
Pipeline(steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 8, 9, 10, 11])),
       

<IPython.core.display.Javascript object>

In [47]:
scores = model_selection.cross_val_score(
    sclf, xtrain, ytrain, cv=10, scoring="f1_weighted"
)
print(
    "Accuracy: %0.4f (+/- %0.4f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "SVC_clf", scores)
)

Fitting 4 classifiers...
Fitting classifier1: randomforestclassifier (1/4)
RandomForestClassifier(max_depth=30, min_samples_leaf=10, n_estimators=150,
                       random_state=42)
Fitting classifier2: xgbclassifier (2/4)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=20, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Fitting classifier3: pipeline (3/4)
Pipeline(steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 8, 9, 10, 11])),
       

Fitting 4 classifiers...
Fitting classifier1: randomforestclassifier (1/4)
RandomForestClassifier(max_depth=30, min_samples_leaf=10, n_estimators=150,
                       random_state=42)
Fitting classifier2: xgbclassifier (2/4)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=20, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Fitting classifier3: pipeline (3/4)
Pipeline(steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 8, 9, 10, 11])),
       

Fitting 4 classifiers...
Fitting classifier1: randomforestclassifier (1/4)
RandomForestClassifier(max_depth=30, min_samples_leaf=10, n_estimators=150,
                       random_state=42)
Fitting classifier2: xgbclassifier (2/4)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=20, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Fitting classifier3: pipeline (3/4)
Pipeline(steps=[('columnselector',
                 ColumnSelector(cols=[0, 1, 3, 8, 9, 10, 11])),
       

<IPython.core.display.Javascript object>

4.3.Softvoting classifier
===

In [49]:
weakmodles = [rf_12, xgb_model_12, LogisticRegression(random_state=0), GaussianNB()]
estimators = [
    ("rf_12", rf_12),
    ("xgb_model_12", xgb_model_12),
    ("LogisticRegression", LogisticRegression(random_state=0)),
    ("GaussianNB", GaussianNB()),
    ("rf_12_pip", rf_12_pip),
    ("xgb_model_12_pip", xgb_model_12_pip),
]

<IPython.core.display.Javascript object>

In [50]:
votingclf2 = VotingClassifier(estimators=estimators, voting="soft")
votingclf2.fit(xtrain, ytrain)

VotingClassifier(estimators=[('rf_12',
                              RandomForestClassifier(max_depth=30,
                                                     min_samples_leaf=10,
                                                     n_estimators=150,
                                                     random_state=42)),
                             ('xgb_model_12',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                                            gpu_id=-1, importance_type='gain',
                                            interaction_constraints='',
                                            learning_rate=0.300000012,
                                            max_del...
                                                             interaction_c

<IPython.core.display.Javascript object>

In [51]:
ypred = votingclf2.predict(xtest.astype(float))

print("accuracy \t", m.accuracy_score(ytest, ypred))
confmatrx = pd.DataFrame(
    m.confusion_matrix(ytest, ypred),
    columns=["Normal", "Prediabetes", "diabetes"],
    index=["Normal", "Prediabetes", "diabetes"],
)
confmatrx.head()

accuracy 	 0.73


,Normal,Prediabetes,diabetes
Normal,149,51,0
Prediabetes,63,126,11
diabetes,4,33,163


<IPython.core.display.Javascript object>

In [52]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.69      0.74      0.72       200
           1       0.60      0.63      0.61       200
           2       0.94      0.81      0.87       200

    accuracy                           0.73       600
   macro avg       0.74      0.73      0.73       600
weighted avg       0.74      0.73      0.73       600



<IPython.core.display.Javascript object>

In [53]:
scores = model_selection.cross_val_score(
    votingclf2, xtrain, ytrain, cv=10, scoring="accuracy"
)
print(
    "Accuracy: %0.4f (+/- %0.4f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "SVC_clf", scores)
)

Accuracy: 0.7526 (+/- 0.0380) [SVC_clf] 
 [[0.64455253 0.73657588 0.75466926 0.77388597 0.7793345  0.78069663
 0.76863203 0.76123759 0.76571317 0.76045923]]


<IPython.core.display.Javascript object>

In [54]:
scores = model_selection.cross_val_score(
    votingclf2, xtrain, ytrain, cv=10, scoring="precision_weighted"
)
print(
    "precision: %0.4f (+/- %0.4f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "SVC_clf", scores)
)

precision: 0.7503 (+/- 0.0352) [SVC_clf] 
 [[0.6502968  0.73532183 0.75224161 0.77043597 0.77586626 0.77609906
 0.76253697 0.75701411 0.76446455 0.75840535]]


<IPython.core.display.Javascript object>

In [55]:
scores = model_selection.cross_val_score(
    votingclf2, xtrain, ytrain, cv=10, scoring="recall_weighted"
)
print(
    "recall: %0.4f (+/- %0.4f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "SVC_clf", scores)
)

recall: 0.7526 (+/- 0.0380) [SVC_clf] 
 [[0.64455253 0.73657588 0.75466926 0.77388597 0.7793345  0.78069663
 0.76863203 0.76123759 0.76571317 0.76045923]]


<IPython.core.display.Javascript object>

In [56]:
scores = model_selection.cross_val_score(
    votingclf2, xtrain, ytrain, cv=10, scoring="f1_weighted"
)
print(
    "Accuracy: %0.4f (+/- %0.4f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "SVC_clf", scores)
)

Accuracy: 0.7489 (+/- 0.0397) [SVC_clf] 
 [[0.63605908 0.73207258 0.75153257 0.7704589  0.77731033 0.77767858
 0.76431101 0.75734653 0.76379727 0.7580235 ]]


<IPython.core.display.Javascript object>

4.4.HardVoting
===

In [37]:
votingclf2 = VotingClassifier(estimators=estimators, voting="hard")
votingclf2.fit(xtrain, ytrain)

VotingClassifier(estimators=[('rf_12',
                              RandomForestClassifier(max_depth=30,
                                                     min_samples_leaf=10,
                                                     n_estimators=150,
                                                     random_state=42)),
                             ('xgb_model_12',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                                            gpu_id=-1, importance_type='gain',
                                            interaction_constraints='',
                                            learning_rate=0.300000012,
                                            max_del...,
                                            min_child_weight=1, missing=n

<IPython.core.display.Javascript object>

In [38]:
ypred = votingclf2.predict(xtest.astype(float))

print("accuracy \t", m.accuracy_score(ytest, ypred))
confmatrx = pd.DataFrame(
    m.confusion_matrix(ytest, ypred),
    columns=["Normal", "Prediabetes", "diabetes"],
    index=["Normal", "Prediabetes", "diabetes"],
)
confmatrx.head()

accuracy 	 0.745


,Normal,Prediabetes,diabetes
Normal,160,39,1
Prediabetes,54,126,20
diabetes,0,39,161


<IPython.core.display.Javascript object>

In [39]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.75      0.80      0.77       200
           1       0.62      0.63      0.62       200
           2       0.88      0.81      0.84       200

    accuracy                           0.74       600
   macro avg       0.75      0.75      0.75       600
weighted avg       0.75      0.74      0.75       600



<IPython.core.display.Javascript object>

In [40]:
scores = model_selection.cross_val_score(
    votingclf2, xtrain, ytrain, cv=10, scoring="accuracy"
)
print(
    "Accuracy: %0.4f (+/- %0.4f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "SVC_clf", scores)
)

Accuracy: 0.7165 (+/- 0.0369) [SVC_clf] 
 [[0.61401297 0.69848703 0.71253602 0.73054755 0.74675793 0.74153458
 0.73487032 0.72136167 0.74184832 0.72275266]]


<IPython.core.display.Javascript object>

In [41]:
scores = model_selection.cross_val_score(
    votingclf2, xtrain, ytrain, cv=10, scoring="precision_weighted"
)
print(
    "precision: %0.4f (+/- %0.4f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "SVC_clf", scores)
)

precision: 0.7219 (+/- 0.0351) [SVC_clf] 
 [[0.62486382 0.7038619  0.71892722 0.73475415 0.75268389 0.74527056
 0.73563173 0.72390087 0.74675516 0.73186627]]


<IPython.core.display.Javascript object>

In [42]:
scores = model_selection.cross_val_score(
    votingclf2, xtrain, ytrain, cv=10, scoring="recall_weighted"
)
print(
    "recall: %0.4f (+/- %0.4f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "SVC_clf", scores)
)

recall: 0.7165 (+/- 0.0369) [SVC_clf] 
 [[0.61401297 0.69848703 0.71253602 0.73054755 0.74675793 0.74153458
 0.73487032 0.72136167 0.74184832 0.72275266]]


<IPython.core.display.Javascript object>

In [43]:
scores = model_selection.cross_val_score(
    votingclf2, xtrain, ytrain, cv=10, scoring="f1_weighted"
)
print(
    "Accuracy: %0.4f (+/- %0.4f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "SVC_clf", scores)
)

Accuracy: 0.7167 (+/- 0.0398) [SVC_clf] 
 [[0.60552926 0.69765651 0.71388999 0.73193422 0.7490811  0.74307919
 0.73486595 0.72174652 0.74352326 0.72524385]]


<IPython.core.display.Javascript object>